# sal-cli

In [ ]:
#| default_exp cli

In [ ]:
#| export
from pathlib import Path
from sal.core import Data
from sal.loaders import xml_file_to_data
from sal.codegen import Sal, FrontMatterInMemoryTemplateLoader, Renderer, JinjaTemplateRenderer, MissingTemplate

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""

destination = '/tmp/something2.txt'

xml = f"""
<to-file to="{destination}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>"""

In [ ]:
#| export 
def render(file, templates):
    try:
        repository = FrontMatterInMemoryTemplateLoader.from_directory(templates)
        renderer = Renderer(repository=repository, renderer=JinjaTemplateRenderer())
        sal = Sal(renderer)

        struct: Data = xml_file_to_data(file)  
        return sal.process(struct)
    except MissingTemplate as e:
        path = Path(templates) / f"{e.name}.jinja2"
        path.write_text(Renderer.DEFAULT_TEMPLATE)
        return render(file, templates)

In [ ]:
#| hide

# create template dir
path = Path('/tmp/templates')
path.mkdir(exist_ok=True)

# create xml file
Path('/tmp/sal.xml').write_text(xml)

# create template files
Path('/tmp/templates/model.jinja2').write_text(model)
Path('/tmp/templates/field.jinja2').write_text(field)
    
# render
render('/tmp/sal.xml', '/tmp/templates')


# test equality
test_eq(
    Path(destination).read_text().strip(), 
"""
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip())
    

We would like to create default templates on the fly when they don't exist

In [ ]:
#| hide

Path('/tmp/templates/model.jinja2').unlink(missing_ok=True)

render('/tmp/sal.xml', '/tmp/templates')

test_eq(Path('/tmp/templates/model.jinja2').exists(), True)
test_eq(Path('/tmp/templates/model.jinja2').read_text(), Renderer.DEFAULT_TEMPLATE)


---

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()